# Learning with a single neuron using Flux.jl

In this notebook, we'll use `Flux` to create a single neuron and teach it to learn, as we did by hand in notebook 10!

### Read in data and process it

Let's start by reading in our data

In [1]:
using CSV
using TextParse
using DataFrames

applecols, applecolnames = TextParse.csvread("data/Apple_Golden_1.dat", '\t')
bananacols, bananacolnames = TextParse.csvread("data/bananas.dat", '\t')

apples = DataFrame(Dict{String, Any}(strip(name)=>col for (name, col) in zip(applecolnames, applecols)))
bananas = DataFrame(Dict{String, Any}(strip(name)=>col for (name, col) in zip(bananacolnames, bananacols)));

and processing it to extract information about the red and green coloring in our images:

In [2]:
col1 = :red
col2 = :green

x_apples  = [ [apples[i, col1], apples[i, col2]] for i in 1:size(apples)[1] ]
x_bananas = [ [bananas[i, col1], bananas[i, col2]] for i in 1:size(bananas)[1] ]

xs = vcat(x_apples, x_bananas)

ys = vcat( zeros(size(x_apples)[1]), ones(size(x_bananas)[1]) );

The input data is in `xs` and the labels (true classifications as bananas or apples) in `ys`.

### Using `Flux.jl`

Now we can load `Flux` to really get going!

In [3]:
using Flux

loaded


We saw in the last notebook that σ is a built-in function in `Flux`.

Another function that is used a lot in neural networks is called `ReLU`; in Julia, the function is called `relu`.

#### Exercise 1

Use the docs to discover what `ReLU` is all about.

`relu.([-3, 3])` returns

A) [-3, 3] <br>
B) [0, 3] <br>
C) [0, 0] <br>
D) [3, 3] <br>

In [4]:
relu.([-3, 3])

2-element Array{Int64,1}:
 0
 3

### Making a single neuron in Flux

Let's use `Flux` to build our neuron with 2 inputs and 1 output:

 <img src="data/single-neuron.png" alt="Drawing" style="width: 500px;"/>

We previously put the two weights in a vector, $\mathbf{w}$. Flux instead puts weights in a $1 \times 2$ matrix (i.e. a matrix with 1 *row* and 2 *columns*). 

Previously, to compute the dot product of $\mathbf{w}$ and $\mathbf{x}$ we had to use either the `dot` function, or we had to transpose the vector $\mathbf{w}$:

```julia
# transpose w
b = w' * x
# or use dot!
b = dot(w, x)
```
If the weights are instead stored in a $1 \times 2$ matrix, `W`, then we can simply multiply `W` and `x` together instead!

We start off with random values for our parameters and data:

In [5]:
W = rand(1, 2)

1×2 Array{Float64,2}:
 0.0316797  0.0684795

In [6]:
x = rand(2)

2-element Array{Float64,1}:
 0.8097524904743549
 0.5948179502909527

Note that the product of `W` and `x` will now be an array (vector) with a single element, rather than a single number:

In [7]:
W * x

1-element Array{Float64,1}:
 0.06638552888249623

This means that our bias `b` is treated as an array when we're using `Flux`:

In [8]:
b = rand(1)

1-element Array{Float64,1}:
 0.026156977654309355

#### Exercise 2

Write a function `mypredict` that will take a single input, array `x` and use `W`, `b`, and built-in `σ` to generate an output prediction (stored as an array). This function defines our neural network!

Hint: This function will look very similar to $f_{\mathbf{w},\mathbf{b}}$ from the last notebook but has changed since our data structures to store our parameters have changed!

In [9]:
function mypredict(x)
    return σ.(W*x .+ b)
end

mypredict (generic function with 1 method)

#### Exercise 3

Define a loss function called `loss`.

`loss` should take two inputs: a vector storing data, `x`, and a vector storing the correct "labels" for that data. `loss` should return the sum of the squares of differences between the predictions and the correct labels.

In [10]:
function loss(x, labels)
    return sum((labels .- mypredict(x)) .^ 2.0)
end

loss (generic function with 1 method)

## Calculating gradients using Flux: backpropagation

For learning, we know that what we need is a way to calculate derivatives of the `loss` function with respect to the parameters `W` and `b`. So far, we have been doing that using finite differences. 

`Flux.jl` instead implements a numerical method called **backpropagation** that calculates gradients (essentially) exactly, in an automatic way, by indirectly applying the rules of calculus.
To do so, it provides a new type of object called "tracked" arrays. These are arrays that store not only their current value, but also information about gradients, which is used by the backpropagation method.

[If you want to understand the maths behind backpropagation, we recommend e.g. [this lecture](https://www.youtube.com/watch?v=i94OvYb6noo).]

To do so, `Flux` provides a function `param` to define such objects that will contain the information for a *param*eter.

Let's start, as usual, by setting up some random initial values for the parameters:

In [11]:
W_data = rand(1, 2)  
b_data = rand(1)

W_data, b_data

([0.293795 0.492515], [0.247829])

We now set up `Flux.jl` objects that will contain these values *and* their derivatives, and allow to propagate
this information around:

In [12]:
W = param(W_data)
b = param(b_data)

Tracked 1-element Array{Float64,1}:
 0.24782853276082428

Here, `param` is a function that `Flux` provides to create an object that represents a parameter of a machine learning model, i.e. an object which has both a value and derivative information, and such that other objects know how to *keep track* of when it is used in an expression.

#### Exercise 4

What type does `W` have?

A) Array (1D) <br>
B) Array (2D) <br>
C) TrackedArray (1D) <br>
D) TrackedArray (2D) <br>
E) Parameter (1D) <br>
F) Parameter (2D) <br>

In [13]:
W

Tracked 1×2 Array{Float64,2}:
 0.293795  0.492515

#### Exercise 5

`W` stores not only its current value, but also has space to store gradient information. You can access the values and gradient of the weights as follows:

```julia
W.data
W.grad
```

At this point, are the values of the weights or the gradient of the weights larger?

A) the values of the weights <br>
B) the gradient of the weights

In [14]:
println(W.data)
println(W.grad)

[0.293795 0.492515]
[0.0 0.0]


#### Exercise 6

For data `x` and `y` where

```julia
x, y = [0.413759, 0.692204], [0.845677]
```
apply the loss function to `x` and `y` to give a new variable `l`. What is the type of `l`? (How many dimensions does it have?)

A) Array (0D) <br>
<font color="red">B) Array (1D) </font><br>
C) TrackedArray (0D) <br>
D) TrackedArray (1D)<br> 
E) Float64<br>
F) Int64<br>

In [15]:
x, y = [0.413759, 0.692204], [0.845677]
y

1-element Array{Float64,1}:
 0.845677

In [16]:
loss(xs[1], ys[1])

0.46775778139994906 (tracked)

### Stochastic gradient descent

We can now use these features to reimplement stochastic gradient descent, following the method we used in the previous notebook, but now using backpropagation!

#### Exercise 7

Modify the code from the previous notebook for stochastic gradient descent to use Flux instead.

In [41]:
# \nabla

LoadError: syntax: "\" is not a unary operator

In [39]:
function stochastic_gradient_descent(L::Function, w, b, xs, ys, N = 1000)
    
    for iter in 1:N
        rand_index = rand(1:size(xs)[1])
        
        println(w, b, L(xs[rand_index], ys[rand_index]), 
            sum(L(x, y) for (x, y) in (xs, ys)))
    end
    return w, b
end

stochastic_gradient_descent (generic function with 2 methods)

### Investigating stochastic gradient descent

Let's look at the values stored in `b` before we run stochastic gradient descent:

In [35]:
b

Tracked 1-element Array{Float64,1}:
 0.24782853276082428

After running `stochastic_gradient_descent`, we find the following:

In [40]:
W_final, b_final = stochastic_gradient_descent(loss, W, b, xs, ys, 100000)

Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)]Flux.Tracker.TrackedReal{Float64}[0.247829 (tracked)]0.2058757301440381 (tracked)0.5041559398124698 (tracked)
Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)]Flux.Tracker.TrackedReal{Float64}[0.247829 (tracked)]0.3114901394747774 (tracked)0.5041559398124698 (tracked)
Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)]Flux.Tracker.TrackedReal{Float64}[0.247829 (tracked)]0.30850217012771475 (tracked)0.5041559398124698 (tracked)
Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)]Flux.Tracker.TrackedReal{Float64}[0.247829 (tracked)]0.2053468210001131 (tracked)0.5041559398124698 (tracked)
Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)]Flux.Tracker.TrackedReal{Float64}[0.247829 (tracked)]0.30747474757281207 (tracked)0.5041559398124698 (tracked)
Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)]Flux.Tracker.Tra

Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)]Flux.Tracker.TrackedReal{Float64}[0.247829 (tracked)]0.2059247889649341 (tracked)0.5041559398124698 (tracked)
Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)]Flux.Tracker.TrackedReal{Float64}[0.247829 (tracked)]0.20803909365983278 (tracked)0.5041559398124698 (tracked)
Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)]Flux.Tracker.TrackedReal{Float64}[0.247829 (tracked)]0.3115512210065849 (tracked)0.5041559398124698 (tracked)
Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)]Flux.Tracker.TrackedReal{Float64}[0.247829 (tracked)]0.2053468210001131 (tracked)0.5041559398124698 (tracked)
Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)]Flux.Tracker.TrackedReal{Float64}[0.247829 (tracked)]0.20885464976599524 (tracked)0.5041559398124698 (tracked)
Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)]Flux.Tracker.Tra

Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)]Flux.Tracker.TrackedReal{Float64}[0.247829 (tracked)]0.30525194534972017 (tracked)0.5041559398124698 (tracked)
Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)]Flux.Tracker.TrackedReal{Float64}[0.247829 (tracked)]0.30815715133354565 (tracked)0.5041559398124698 (tracked)
Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)]Flux.Tracker.TrackedReal{Float64}[0.247829 (tracked)]0.31069612871192226 (tracked)0.5041559398124698 (tracked)
Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)]Flux.Tracker.TrackedReal{Float64}[0.247829 (tracked)]0.3057496959707838 (tracked)0.5041559398124698 (tracked)
Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)]Flux.Tracker.TrackedReal{Float64}[0.247829 (tracked)]0.20924932414202846 (tracked)0.5041559398124698 (tracked)
Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)]Flux.Tracker.T

Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)]Flux.Tracker.TrackedReal{Float64}[0.247829 (tracked)]0.306118675656714 (tracked)0.5041559398124698 (tracked)
Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)]Flux.Tracker.TrackedReal{Float64}[0.247829 (tracked)]0.3096415220042166 (tracked)0.5041559398124698 (tracked)
Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)]Flux.Tracker.TrackedReal{Float64}[0.247829 (tracked)]0.20237655909565946 (tracked)0.5041559398124698 (tracked)
Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)]Flux.Tracker.TrackedReal{Float64}[0.247829 (tracked)]0.3116140083479278 (tracked)0.5041559398124698 (tracked)
Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)]Flux.Tracker.TrackedReal{Float64}[0.247829 (tracked)]0.3079244586127301 (tracked)0.5041559398124698 (tracked)
Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)]Flux.Tracker.Track

Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)]Flux.Tracker.TrackedReal{Float64}[0.247829 (tracked)]0.3065225724844011 (tracked)0.5041559398124698 (tracked)
Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)]Flux.Tracker.TrackedReal{Float64}[0.247829 (tracked)]0.30796910676411926 (tracked)0.5041559398124698 (tracked)
Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)]Flux.Tracker.TrackedReal{Float64}[0.247829 (tracked)]0.20845519870896342 (tracked)0.5041559398124698 (tracked)
Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)]Flux.Tracker.TrackedReal{Float64}[0.247829 (tracked)]0.2114366790816614 (tracked)0.5041559398124698 (tracked)
Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)]Flux.Tracker.TrackedReal{Float64}[0.247829 (tracked)]0.20624727776865256 (tracked)0.5041559398124698 (tracked)
Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)]Flux.Tracker.Tr

...

Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)]Flux.Tracker.TrackedReal{Float64}[0.247829 (tracked)]0.3123689269109356 (tracked)0.5041559398124698 (tracked)
Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)]Flux.Tracker.TrackedReal{Float64}[0.247829 (tracked)]0.30796342183198205 (tracked)0.5041559398124698 (tracked)
Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)]Flux.Tracker.TrackedReal{Float64}[0.247829 (tracked)]0.30504751707896316 (tracked)0.5041559398124698 (tracked)
Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)]Flux.Tracker.TrackedReal{Float64}[0.247829 (tracked)]0.305008608476436 (tracked)0.5041559398124698 (tracked)
Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)]Flux.Tracker.TrackedReal{Float64}[0.247829 (tracked)]0.20297489205878258 (tracked)0.5041559398124698 (tracked)
Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)]Flux.Tracker.Tra

Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)]Flux.Tracker.TrackedReal{Float64}[0.247829 (tracked)]0.30514781039191485 (tracked)0.5041559398124698 (tracked)
Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)]Flux.Tracker.TrackedReal{Float64}[0.247829 (tracked)]0.3122107480064756 (tracked)0.5041559398124698 (tracked)
Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)]Flux.Tracker.TrackedReal{Float64}[0.247829 (tracked)]0.305008608476436 (tracked)0.5041559398124698 (tracked)
Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)]Flux.Tracker.TrackedReal{Float64}[0.247829 (tracked)]0.21312962307160557 (tracked)0.5041559398124698 (tracked)
Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)]Flux.Tracker.TrackedReal{Float64}[0.247829 (tracked)]0.30809546395304876 (tracked)0.5041559398124698 (tracked)
Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)]Flux.Tracker.Tra

Excessive output truncated after 524289 bytes.

Flux.Tracker.TrackedReal{Float64}[0.247829 (tracked)]0.20962228490383694 (tracked)0.5041559398124698 (tracked)
Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)]Flux.Tracker.TrackedReal{Float64}[0.247829 (tracked)]0.20379760754791276 (tracked)0.5041559398124698 (tracked)
Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)]Flux.Tracker.TrackedReal{Float64}[0.247829 (tracked)]0.307376621348899 (tracked)0.5041559398124698 (tracked)
Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)]Flux.Tracker.TrackedReal{Float64}[0.247829 (tracked)]0.3077970485413646 (tracked)0.5041559398124698 (tracked)
Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)]Flux.Tracker.TrackedReal{Float64}[0.247829 (tracked)]0.30818471678203124 (tracked)0.5041559398124698 (tracked)
Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)]Flux.Tracker.TrackedReal{Float64}[0.247829 (tracked)]0.3080373979000284 (tracked)0.50415

(Flux.Tracker.TrackedReal{Float64}[0.293795 (tracked) 0.492515 (tracked)], Flux.Tracker.TrackedReal{Float64}[0.247829 (tracked)])

### we can look at the values of `W_final` and `b_final`, which our machine learned to generate our desired classification.

In [20]:
W_final

UndefVarError: UndefVarError: W_final not defined

In [21]:
b_final

UndefVarError: UndefVarError: b_final not defined

#### Exercise 8

Plot the data and the learned function.

#### Exercise 9

Do this plot every so often as the learning process is proceeding in order to have an animation of the process.

### Automation with Flux.jl

We will need to repeat the above process for a lot of different systems.
Fortunately, `Flux.jl` provides us with tools to automate this!

Flux allows to create a neuron in a simple way:

In [22]:
using Flux

In [23]:
model = Dense(2, 1, σ)

Dense(2, 1, NNlib.σ)

The `2` and `1` refer to the number of inputs and outputs, and the neuron is defined using the $\sigma$ function.

In [24]:
typeof(model)

Dense{typeof(σ),TrackedArray{…,Array{Float64,2}},TrackedArray{…,Array{Float64,1}}}

We have made an object of type `Dense`, defined by `Flux`, with the name `model`. This represents a "dense neural network layer" (see later for more on neural network layers).
The parameters that will be modified during the learning process live *inside* the `model` object.

#### Exercise 10

Investigate which variables live inside the `model` object and what type they are. How does that compare to the call to create the `Dense` object that we started with?

### Model object as a function

We can apply the `model` object to data just as if it were a standard function:

In [25]:
model(rand(2))

Tracked 1-element Array{Float64,1}:
 0.5543600633550765

#### Exercise 11

Prove to yourself that you understand what is going on when we call `model`. Create two arrays `W` and `b` with the same elements as `model.W` and `model.b`. Use `W` and `b` to generate the same answer that you get when we call `model([.5, .5])`.

### Using Flux

We now need to provide Flux with three pieces of information: 

1. A loss function
2. Some training data
3. An optimization method

### Loss functions

Flux has various loss functions built in, for example the mean-squared error (`mse`) that we have been using:

In [26]:
loss(x, y) = Flux.mse(model(x), y)

loss (generic function with 1 method)

Another common one is the cross entropy, `Flux.crossentropy`.

### Data

The data can take a couple of different forms. 
One form is a single **iterator**, consisting of pairs $(x, y)$ of data and labels.
We can achieve this with `zip`.

#### Exercise 12

Use `zip` to "zip together" `xs` and `ys`. Then use the `collect` function to check what `zip` actually does.

### Optimization routine

Now we need to tell Flux what kind of optimization routine to use. It has several built in; the standard stochastic gradient descent algorithm that we have been using is called `SGD`. We must pass it two things: a list of parameter objects which will be modified by the optimization routine, and a step size:

In [27]:
opt = SGD([model.W, model.b], 0.01)
# give a list of the parameters that will be modified

#43 (generic function with 1 method)

The gradient calculations and parameter updates will be carried out by this optimizer function; we do not see those details, but if you are curious, you can, of course, look at the `Flux.jl` source code!

### Training

We now have all the pieces in place to actually **train** our model (a single neuron) on the data. 
"Training" refers to using pre-labeled data to learn the function that relates the input data to the desired output data given by the labels.

`Flux` provides the function `train!`, which performs a single pass through the data and does a single step of optimization using the partial cost function for each data point:

In [28]:
Flux.train!(loss, data, opt)

UndefVarError: UndefVarError: data not defined

We can then just repeat this several times to train the network more and coax it towards the minimum of the cost function:

In [29]:
for i in 1:100
    Flux.train!(loss, data, opt)
end

UndefVarError: UndefVarError: data not defined

Now let's look at the parameters after training:

In [30]:
model.W

Tracked 1×2 Array{Float64,2}:
 0.344513  -0.0217488

In [31]:
model.b

Tracked 1-element Array{Float64,1}:
 0.0

Instead of writing out a list of parameters to modify, `Flux` provides the function `params`, which extracts all available parameters from a model:

In [32]:
opt = SGD(params(model), 0.01)

#43 (generic function with 1 method)

In [33]:
params(model)

2-element Array{Any,1}:
 Flux.Tracker.TrackedReal{Float64}[0.344513 (tracked) -0.0217488 (tracked)]
 Flux.Tracker.TrackedReal{Float64}[0.0 (tracked)]                          

## Adding more features

#### Exercise 13

So far we have just used two features, red and green. 

(i) Add a third feature, blue. Plot the new data.

(ii) Train a neuron with 3 inputs and 1 output on the data.

(iii) Can you find a good way to visualize the result?